# Title
[]()

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

# Initialize Dictionaries

In [9]:
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()


# Loading documents

In [19]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory

In [3]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../data/Vancouver apartments condos townhomes.txt")
documents = loader.load()
documents

[Document(page_content='City of Vancouver\nhttps://vancouver.ca/home-property-development/apartments-condos-and-townhomes.aspx\n\nApartments, condos, and townhomes\n\nWe do not provide waste collection services to most multi-unit buildings.\n\nHowever, we do provide information and resources to building owners and managers to manage waste at apartments, condos, and townhomes.\n\nWaste Management Canada collects your recycling on behalf of Recycle BC External website, opens in new tab\nPhone Waste Management Canada at 604-282-7961 for questions and issues about your recycling service.', metadata={'source': '../data/Vancouver apartments condos townhomes.txt'})]

In [4]:
from langchain.document_loaders import TextLoader
loaders = [
    TextLoader("../data/Vancouver apartments condos townhomes.txt"),
    TextLoader("../data/Vancouver single family homes and duplexes.txt")
]

documents = []
for loader in loaders:
    documents.extend(loader.load())

documents

[Document(page_content='City of Vancouver\nhttps://vancouver.ca/home-property-development/apartments-condos-and-townhomes.aspx\n\nApartments, condos, and townhomes\n\nWe do not provide waste collection services to most multi-unit buildings.\n\nHowever, we do provide information and resources to building owners and managers to manage waste at apartments, condos, and townhomes.\n\nWaste Management Canada collects your recycling on behalf of Recycle BC External website, opens in new tab\nPhone Waste Management Canada at 604-282-7961 for questions and issues about your recycling service.', metadata={'source': '../data/Vancouver apartments condos townhomes.txt'}),
 Document(page_content='City of Vancouver\nhttps://vancouver.ca/home-property-development/single-family-homes-and-duplexes.aspx\n\nSingle family homes and duplexes\nWe collect residential garbage, food scraps, and yard waste for single family homes and duplexes.\n\nYour household waste collection schedule is based on your location

In [15]:
site_key = 1 # City of Vancouver Website
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings_dict[site_key] = OpenAIEmbeddings()
db_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
retriever_dict[site_key] = db_dict[site_key].as_retriever()
tools_dict['CoV'] = create_retriever_tool(
    retriever, 
    "search_CoV",
    "Searches and returns documents regarding waste and recycling in the City of Vancouver."
)
tools = [tools_dict['CoV']]
tools

ImportError: Could not import tiktoken python package. This is needed in order to for OpenAIEmbeddings. Please install it with `pip install tiktoken`.

## installed tiktoken, then refreshed kernel

In [5]:
site_key = 1 # City of Vancouver Website
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings_dict[site_key] = OpenAIEmbeddings()
db_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
retriever_dict[site_key] = db_dict[site_key].as_retriever()
tools_dict['CoV'] = create_retriever_tool(
    retriever, 
    "search_CoV",
    "Searches and returns documents regarding waste and recycling in the City of Vancouver."
)
tools = [tools_dict['CoV']]
tools

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

## `pip install faiss-cpu` then refreshed kernel

In [13]:
site_key = 'CoV' # City of Vancouver Website
iteration = 1
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings_dict[site_key] = OpenAIEmbeddings()
db_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
retriever_dict[site_key] = db_dict[site_key].as_retriever()
tool_dict[iteration] = create_retriever_tool(
    retriever_dict['CoV'], 
    "search_CoV",
    "Searches and returns documents regarding waste and recycling in the City of Vancouver."
)
tools = [tool_dict[1]]
tools

[Tool(name='search_CoV', description='Searches and returns documents regarding waste and recycling in the City of Vancouver.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000016D4E517ED0>, search_type='similarity', search_kwargs={})>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000016D4E517ED0>, search_type='similarity', search_kwargs={})>)]

In [27]:
retriever_dict['CoV']

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000016D4E517ED0>, search_type='similarity', search_kwargs={})

# Build agent w/ memory
It looks like since the `tools` variable already can contain multiple retrievers, I don't need to [Dynamically select from multiple retrievers](https://python.langchain.com/docs/use_cases/question_answering/how_to/multi_retrieval_qa_router)

In [ ]:
retriever_infos = [
    {
        "name": "state of the union", 
        "description": "Good for answering questions about the 2023 State of the Union address", 
        "retriever": sou_retriever
    }
]

In [21]:
llm = ChatOpenAI(
    temperature = 0,
    openai_organization=os.environ['openai_organization'],
    openai_api_key=os.environ['openai_api_key'],
    )
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, db_dict[site_key].as_retriever(), memory=memory)
# chain = MultiRetrievalQAChain.from_retrievers(llm, retriever_infos, verbose=True, memory=memory)
query = "I live in a house in Vancouver. How do I separate my waste?"
result = qa({"question": query})
result

{'question': 'I live in a house in Vancouver. How do I separate my waste?',
 'chat_history': [HumanMessage(content='I live in a house in Vancouver. How do I separate my waste?', additional_kwargs={}, example=False),
  AIMessage(content="As a resident of a single-family home or duplex in Vancouver, you can separate your waste into different categories. Here's how you can do it:\n\n1. Garbage: Place non-recyclable and non-compostable waste in your regular garbage bin. Make sure to tie the garbage bags securely.\n\n2. Recycling: GFL Environmental (formerly Smithrite Disposal Ltd) handles recycling collection on behalf of Recycle BC. You can contact them at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service. They provide blue bins and recycling bags for your recyclable materials like paper, cardboard, plastic containers, metal cans, and glass bottles.\n\n3. Food Scraps: Vancouver collects food scraps for composting. You can use a separat

In [26]:
print(result)

{'question': 'I live in a house in Vancouver. How do I separate my waste?', 'chat_history': [HumanMessage(content='I live in a house in Vancouver. How do I separate my waste?', additional_kwargs={}, example=False), AIMessage(content="As a resident of a single-family home or duplex in Vancouver, you can separate your waste into different categories. Here's how you can do it:\n\n1. Garbage: Place non-recyclable and non-compostable waste in your regular garbage bin. Make sure to tie the garbage bags securely.\n\n2. Recycling: GFL Environmental (formerly Smithrite Disposal Ltd) handles recycling collection on behalf of Recycle BC. You can contact them at 604-282-7966 or email srrecycle@gflenv.com for any questions or issues regarding your recycling service. They provide blue bins and recycling bags for your recyclable materials like paper, cardboard, plastic containers, metal cans, and glass bottles.\n\n3. Food Scraps: Vancouver collects food scraps for composting. You can use a separate g

# *End of Page*